In [1]:
#installations
!pip install transformers
!pip install torch-geometric
!pip install pyarabic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 2.6 MB/s eta 0:00:00


In [2]:
import spacy
import pandas as pd
import pyarabic.araby as araby
import unicodedata as ud
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from torch_geometric.nn import GATConv
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
!git clone https://github.com/aub-mind/arabert
!pip install -r arabert/requirements.txt
import tensorflow as tf # to create the model
from tensorflow import keras # to create the model
import numpy as np
import io
!pip install arabert
!pip install transformers[tf-cpu]
from arabert.preprocess import ArabertPreprocessor

Cloning into 'arabert'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 600 (delta 38), reused 45 (delta 30), pack-reused 535
Receiving objects: 100% (600/600), 9.14 MiB | 27.13 MiB/s, done.
Resolving deltas: 100% (339/339), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186459 sha256=7a746800de5802bdc3e68efd6c37e1c8c400d6185380e91c556347aa1b3adb5a
  Stored in directory: /root/.cache/pip/wheels/10/f0/fd/4813b1177405693e8da9cdea839f0fb64fde161380e058c827
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.2/198.2 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [72]:
!pip uninstall keras

Found existing installation: keras 2.14.0
Uninstalling keras-2.14.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/keras-2.14.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/keras/*
Proceed (Y/n)? y
  Successfully uninstalled keras-2.14.0


In [4]:
!pip install keras==2.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.14.0 requires keras<2.15,>=2.14.0, but you have keras 2.6.0 which is incompatible.
tensorflow-cpu 2.14.0 requires keras<2.15,>=2.14.0, but you have keras 2.6.0 which is incompatible.


In [6]:
# Specify the pre-trained AraBERT model you want to use
model_name = "aubmindlab/bert-base-arabertv2"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Create an instance of the ArabertPreprocessor
arabert_prep = ArabertPreprocessor(model_name=model_name)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


100%|██████████| 241M/241M [00:24<00:00, 9.83MiB/s]


[2023-10-27 17:31:48,189 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [7]:
# Read training data
train_dataset = pd.read_csv('/content/My_AR_HOTE_SB1_TRAIN (1).csv')
# Read testing data
test_dataset = pd.read_csv('/content/My_AR_HOTE_SB1_TEST (1).csv')

In [8]:
train_dataset.shape

(10521, 8)

In [9]:
test_dataset.shape

(2606, 8)

In [10]:
train_dataset.head()

,/sentence/#id,/sentence/@id,Category,/sentence/aspectTerms/aspectTerm/@from,Polarity,Term,/sentence/aspectTerms/aspectTerm/@to,Text
0,1,456,LOCATION#GENERAL,31.0,positive,موقع,35.0,أنصح بالنوم وليس تناول الطعام موقع مثالي للإق...
1,2,456,ROOMS#GENERAL,5.0,positive,الغرفة,11.0,كانت الغرفة ممتازة وكذلك الموظفون وبوفيه الإفط...
2,2,456,SERVICE#GENERAL,25.0,positive,الموظفون,33.0,كانت الغرفة ممتازة وكذلك الموظفون وبوفيه الإفط...
3,2,456,FOOD_DRINKS#QUALITY,35.0,positive,بوفيه الإفطار,48.0,كانت الغرفة ممتازة وكذلك الموظفون وبوفيه الإفط...
4,2,456,FOOD_DRINKS#PRICES,67.0,negative,وجبة العشاء,78.0,كانت الغرفة ممتازة وكذلك الموظفون وبوفيه الإفط...


In [11]:
test_dataset.head()

,/sentence/#id,/sentence/@id,Category,/sentence/aspectTerms/aspectTerm/@from,Polarity,Term,/sentence/aspectTerms/aspectTerm/@to,Text
0,1,287,ROOMS_AMENITIES#GENERAL,102.0,negative,الكماليات,111.0,كانت رحلتي لهذا المرفق موفقه ، لكن كانت بعض ال...
1,2,287,FACILITIES#GENERAL,84.0,positive,المنتجع,91.0,واقترح أن يوضع حاجز بين البلكونات في الغرف وال...
2,3,1039,ROOMS_AMENITIES#GENERAL,11.0,negative,الاجهزة الكهربائية,29.0,رضى النزيل الاجهزة الكهربائية غير سليمة الشاشا...
3,3,1039,ROOMS_AMENITIES#GENERAL,40.0,negative,الشاشات,47.0,رضى النزيل الاجهزة الكهربائية غير سليمة الشاشا...
4,3,1039,LOCATION#GENERAL,68.0,positive,الغرفه,74.0,رضى النزيل الاجهزة الكهربائية غير سليمة الشاشا...


In [12]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10521 entries, 0 to 10520
Data columns (total 8 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   /sentence/#id                           10521 non-null  int64  
 1   /sentence/@id                           10521 non-null  int64  
 2   Category                                10509 non-null  object 
 3   /sentence/aspectTerms/aspectTerm/@from  10509 non-null  float64
 4   Polarity                                10509 non-null  object 
 5   Term                                    9620 non-null   object 
 6   /sentence/aspectTerms/aspectTerm/@to    10509 non-null  float64
 7   Text                                    10521 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 657.7+ KB


In [13]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2606 entries, 0 to 2605
Data columns (total 8 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   /sentence/#id                           2606 non-null   int64  
 1   /sentence/@id                           2606 non-null   int64  
 2   Category                                2604 non-null   object 
 3   /sentence/aspectTerms/aspectTerm/@from  2604 non-null   float64
 4   Polarity                                2604 non-null   object 
 5   Term                                    2372 non-null   object 
 6   /sentence/aspectTerms/aspectTerm/@to    2604 non-null   float64
 7   Text                                    2606 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 163.0+ KB


In [13]:
train_X= train_dataset.copy()
#remove polarity coulmn
train_X=train_X.drop(['/sentence/#id','/sentence/@id','Category','/sentence/aspectTerms/aspectTerm/@from','Polarity','/sentence/aspectTerms/aspectTerm/@to'],axis=1)
train_Y=train_dataset.pop('Polarity')
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10521 entries, 0 to 10520
Data columns (total 7 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   /sentence/#id                           10521 non-null  int64  
 1   /sentence/@id                           10521 non-null  int64  
 2   Category                                10509 non-null  object 
 3   /sentence/aspectTerms/aspectTerm/@from  10509 non-null  float64
 4   Term                                    9620 non-null   object 
 5   /sentence/aspectTerms/aspectTerm/@to    10509 non-null  float64
 6   Text                                    10521 non-null  object 
dtypes: float64(2), int64(2), object(3)
memory usage: 575.5+ KB


In [11]:
train_X.head()

,Term,Text
0,موقع,أنصح بالنوم وليس تناول الطعام موقع مثالي للإق...
1,الغرفة,كانت الغرفة ممتازة وكذلك الموظفون وبوفيه الإفط...
2,الموظفون,كانت الغرفة ممتازة وكذلك الموظفون وبوفيه الإفط...
3,بوفيه الإفطار,كانت الغرفة ممتازة وكذلك الموظفون وبوفيه الإفط...
4,وجبة العشاء,كانت الغرفة ممتازة وكذلك الموظفون وبوفيه الإفط...


In [14]:
test_X= test_dataset.copy()
#remove polarity coulmn
test_X=test_X.drop(['/sentence/#id','/sentence/@id','Category','/sentence/aspectTerms/aspectTerm/@from','Polarity','/sentence/aspectTerms/aspectTerm/@to'],axis=1)
test_Y=test_dataset.pop('Polarity')

In [13]:
test_X.head()

,Term,Text
0,الكماليات,كانت رحلتي لهذا المرفق موفقه ، لكن كانت بعض ال...
1,المنتجع,واقترح أن يوضع حاجز بين البلكونات في الغرف وال...
2,الاجهزة الكهربائية,رضى النزيل الاجهزة الكهربائية غير سليمة الشاشا...
3,الشاشات,رضى النزيل الاجهزة الكهربائية غير سليمة الشاشا...
4,الغرفه,رضى النزيل الاجهزة الكهربائية غير سليمة الشاشا...


In [14]:
train_Y.head()

0    positive
1    positive
2    positive
3    positive
4    negative
Name: Polarity, dtype: object

In [15]:
test_Y.head()

0    negative
1    positive
2    negative
3    negative
4    positive
Name: Polarity, dtype: object

In [15]:
input_shape= [train_X.shape[1]]
input_shape

[2]

In [16]:
print(type(train_X))

<class 'pandas.core.frame.DataFrame'>


In [17]:
with open('/content/My_AR_HOTE_SB1_TRAIN (1).csv', 'rb') as file:
    excel_data = file.read()

# Create a BytesIO object and pass the data to it
train_dataset = pd.read_csv(io.BytesIO(excel_data))

# Now you can work with the 'train_dataset' DataFrame
train_dataset.head()

,/sentence/#id,/sentence/@id,Category,/sentence/aspectTerms/aspectTerm/@from,Polarity,Term,/sentence/aspectTerms/aspectTerm/@to,Text
0,1,456,LOCATION#GENERAL,31.0,positive,موقع,35.0,أنصح بالنوم وليس تناول الطعام موقع مثالي للإق...
1,2,456,ROOMS#GENERAL,5.0,positive,الغرفة,11.0,كانت الغرفة ممتازة وكذلك الموظفون وبوفيه الإفط...
2,2,456,SERVICE#GENERAL,25.0,positive,الموظفون,33.0,كانت الغرفة ممتازة وكذلك الموظفون وبوفيه الإفط...
3,2,456,FOOD_DRINKS#QUALITY,35.0,positive,بوفيه الإفطار,48.0,كانت الغرفة ممتازة وكذلك الموظفون وبوفيه الإفط...
4,2,456,FOOD_DRINKS#PRICES,67.0,negative,وجبة العشاء,78.0,كانت الغرفة ممتازة وكذلك الموظفون وبوفيه الإفط...


In [18]:
train_text= train_dataset['Text']
train_term= train_dataset['Term']
train_polarity= train_dataset['Polarity']


In [19]:
#test_dataset = pd.read_csv('/content/My_AR_HOTE_SB1_TEST (1).csv')
#test_dataset.head()

with open('/content/My_AR_HOTE_SB1_TEST (1).csv', 'rb') as file:
    excel_data = file.read()

# Create a BytesIO object and pass the data to it
test_dataset = pd.read_csv(io.BytesIO(excel_data))

# Now you can work with the 'test_dataset' DataFrame
test_dataset.head()

,/sentence/#id,/sentence/@id,Category,/sentence/aspectTerms/aspectTerm/@from,Polarity,Term,/sentence/aspectTerms/aspectTerm/@to,Text
0,1,287,ROOMS_AMENITIES#GENERAL,102.0,negative,الكماليات,111.0,كانت رحلتي لهذا المرفق موفقه ، لكن كانت بعض ال...
1,2,287,FACILITIES#GENERAL,84.0,positive,المنتجع,91.0,واقترح أن يوضع حاجز بين البلكونات في الغرف وال...
2,3,1039,ROOMS_AMENITIES#GENERAL,11.0,negative,الاجهزة الكهربائية,29.0,رضى النزيل الاجهزة الكهربائية غير سليمة الشاشا...
3,3,1039,ROOMS_AMENITIES#GENERAL,40.0,negative,الشاشات,47.0,رضى النزيل الاجهزة الكهربائية غير سليمة الشاشا...
4,3,1039,LOCATION#GENERAL,68.0,positive,الغرفه,74.0,رضى النزيل الاجهزة الكهربائية غير سليمة الشاشا...


In [20]:
test_text= test_dataset['Text']
test_term= test_dataset['Term']
test_polarity= test_dataset['Polarity']

In [21]:
!pip install transformers[tf-gpu]

In [22]:
!git clone https://github.com/aub-mind/arabert
!pip install -r arabert/requirements.txt

fatal: destination path 'arabert' already exists and is not an empty directory.


In [23]:
from arabert.preprocess import ArabertPreprocessor

In [24]:
model_name = "aubmindlab/bert-base-arabertv2"
#model_name="aubmindlab/bert-base-arabertv01"
arabert_prep = ArabertPreprocessor(model_name=model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

[2023-10-27 17:34:00,705 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [25]:
train_text = train_text.apply(arabert_prep.preprocess)

In [26]:
train_text[0]

'أنصح ب+ ال+ نوم و+ ليس تناول ال+ طعام موقع مثالي ل+ ال+ إقام +ة قبل رحل +ة طيران مبكر +ة .'

In [27]:
train_term = train_term.apply(arabert_prep.preprocess)

In [28]:
train_term[0]

'موقع'

In [29]:
train_text_tokenized = []
train_term_tokenized = []

def text_tokenization(text):
  arr=[]
  for i in text:
    inputs = tokenizer.encode_plus(i, return_tensors='pt',max_length=512)
    arr.append(inputs)
  return arr

train_text_tokenized = text_tokenization(train_text)
train_term_tokenized = text_tokenization(train_term)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [46]:
train_text_outputs = []
train_term_outputs = []

def to_model(text):
  arr=[]
  for i in text:
    with torch.no_grad():
      outputs = model(**i)
      arr.append(outputs)
  return arr

train_text_outputs = to_model(train_text_tokenized)
train_term_outputs = to_model(train_term_tokenized)

In [47]:
train_text_outputs[0]

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.0232, -0.2530,  0.1050,  ...,  0.0687, -0.0856,  0.1506],
         [ 0.0869, -0.3503,  0.0472,  ...,  0.0863, -0.2220,  0.6815],
         [ 0.3448, -0.5392,  0.0919,  ...,  0.1072, -0.1241, -0.5039],
         ...,
         [-0.2129, -0.0328,  0.2112,  ...,  0.1492, -0.6822,  0.0597],
         [ 0.3946, -0.4122, -0.2010,  ...,  0.1715,  0.1863,  0.5410],
         [-0.2131, -0.0326,  0.2109,  ...,  0.1495, -0.6824,  0.0595]]]), pooler_output=tensor([[-7.7621e-02,  9.9987e-01, -9.9955e-01,  8.8562e-01, -3.0188e-01,
          9.9681e-01,  8.3545e-01,  2.5358e-01,  5.6882e-02,  3.7716e-01,
         -9.9979e-01,  9.9972e-01,  1.7206e-01, -9.8474e-01, -1.0492e-01,
          2.2418e-01, -5.9238e-02,  1.0000e+00,  9.9989e-01, -9.9967e-01,
         -9.9976e-01, -9.9784e-01, -1.8793e-01,  8.1828e-01,  7.4957e-02,
         -5.3736e-02, -9.9875e-01, -9.5511e-02,  5.2730e-02,  9.9918e-01,
         -9.7136e-01,  2.1639e-03,  

In [30]:
train_text_tokenized[0]

{'input_ids': tensor([[   33, 18603,     2,    20,  4154,    29,   682,  2345,    20,  3735,
          1099,  5925,     6,    20,  1567,    12,   554,  1975,    12,  2463,
          3072,    12,    48,    34]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [48]:






train_text_embeddings = []
train_term_embeddings = []

def embedding(text):
  arr=[]
  for i in text:
    embeddings = i['last_hidden_state']
    arr.append(embeddings)
  return arr

train_text_embeddings = embedding(train_text_outputs)
train_term_embeddings = embedding(train_term_outputs)






In [49]:
size= len(train_text_embeddings)
print(str(size))

10521


In [50]:
size= len(train_term_embeddings)
print(str(size))

10521


In [51]:
#test dataset embedding
test_text = test_text.apply(arabert_prep.preprocess)
test_term = test_term.apply(arabert_prep.preprocess)

In [52]:
test_text_tokenized = []
test_term_tokenized = []

def text_tokenization(text):
  arr=[]
  for i in text:
    inputs = tokenizer.encode_plus(i, return_tensors='pt',max_length=512)
    arr.append(inputs)
  return arr

test_text_tokenized = text_tokenization(test_text)
test_term_tokenized = text_tokenization(test_term)

In [53]:
test_text_outputs = []
test_term_outputs = []

def to_model(text):
  arr=[]
  for i in text:
    with torch.no_grad():
      outputs = model(**i)
      arr.append(outputs)
  return arr

test_text_outputs = to_model(test_text_tokenized)
test_term_outputs = to_model(test_term_tokenized)

In [54]:
test_text_embeddings = []
test_term_embeddings = []

def embedding(text):
  arr=[]
  for i in text:
    embeddings = i['last_hidden_state']
    arr.append(embeddings)
  return arr

test_text_embeddings = embedding(test_text_outputs)
test_term_embeddings = embedding(test_term_outputs)

In [55]:
size= len(test_text_embeddings)
print(str(size))

2606


In [56]:
size= len(test_term_embeddings)
print(str(size))

2606
